In [16]:
import numpy as np
from scipy.optimize import minimize_scalar

### Python script deel 1: Berekening
In dit deel van het script wordt de berekening van de poer uitgevoerd. Deze berekning roept verschillende functies aan die ook zijn gedefinieerd in dit script. Uiteindelijk worden hiermee de berekening en de controleberekening uitgevoerd. Deze worden aangeroepen in het script 'Optimalisatie'. 

In het vervolg wordt per functie toegelicht wat die precies doet en hoe die dat uitvoert.

#### Betondekking

De betondekking is de dikte van het beton dat de wapening bedekt. De dekking heeft als doel de wapening en daarmee de poer te beschermen tegen omgevingsinvloeden. Een voldoende dikke betondekking is daarom noodzakelijk. In de functie hieronder wordt de minimaal benodigde betondekking berekend, deze is afhankelijk van de diameter van de wapeningsstaven (dstaaf). De uiteindelijk toegepaste dekking kan worden gekozen, dit kan bijvoorbeeld gedaan worden omdat de poer niet voldoet aan de eisen van scheurvorming. In de tweede functie wordt de gekozen betondekking gecontroleerd op de minimaal benodigde dekking.

In [21]:
def betondekking(dstaaf, constructieklasse):
    CK = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
    cmindur = [10, 15, 20, 25, 30, 35]
    ck = CK.index(constructieklasse)
    cmindur = cmindur[ck]                    #EC2 tabel 4.4N, afhankelijk van de constructieklasse van het beton
    cminb = dstaaf                            
    dcdev = 5                                
    cmin = max([cminb, cmindur, 10]) + dcdev #EC2 vgl 4.1 en 4.2
    k1 = cmindur + 10                        #minimale extra marge bij storten tegen oneffen oppervlak (EC2 art. 4.4.1.3 (4))
    cnom = max([cmin, k1])
    return cnom

def testdekking(c, cnom, dstaaf):
    if c >= cnom and c <= 2 * dstaaf:
        return True
    else:
        return False

#### Staafwerkmodel

De drukschoor en trekstaaf in de poer kunnen worden  versimpeld als inwendig staafwerk. De berekeningen op spanning in de poer zal dan ook worden gedaan aan de hand van het staafwerkmodel. Het staafwerk bestaat als twee diagonale driehoeken die elkaar snijden in het midden van de poer. Voor dit model moeten 4 elementen getoetst worden. Knoop a bij de aansluiting tussen de kolom en de poer. Knoop B bij de aansluiting tussen de poer en de funderingspaal. De drukschoor tussen knoop a en knoop b. En als laatste de trekstaaf tussen de knopen b, deze trekkracht wordt opgevangen door de wapening. 

##### Knoop a
Knoop a bevindt zich bovenin de poer direct onder de kolom. De spanningen rond knoop a zijn afhankelijk van de belasting en de afmetingen van de kolom. De inwendig hoek van het staafwerk (theta) beïnvloedt de waarde van de drukspanning in de drukstaaf van het staafwerk. De maximaal teolaatbare spanning in knoop a is afhankelijk van de druksterkte van het beton. Omdat knoop a 3-assige druk ondervindt mag de maimale drukspanning verhoogd worden. Deze spanning kan worden berekend met behulp van EC2 vgl. 3.15 en 3.25. Met de functie 'testA()' wordt de optredende spanning getoets aan de maximaal toelaatbare spanning.

In [23]:
def knoopA(Dk, ULS, theta):
    a0 = 0.5 * Dk
    F0 = 0.25 * ULS
    a3 = 0.5 * Dk / np.tan(theta)
    F3 = F0 / np.tan(theta)
    a1a = np.sqrt(a0**2 + a3**2)
    F1 = np.sqrt(F0**2 + F3**2)

    sigmaEd1 = F1 * 1000 / (Dk * a1a)           #rekenwaarde enkelassige spanning gebruikt in maxA():
    sigmaEd0 = 2 * F0 * 1000 / (Dk * a0)        #Rekenwaarde spanning, verdubbeld omdat spanningen overlappen.
    return sigmaEd1, sigmaEd0, a1a

def maxA(sigmaEd1, fck):                        #toelaabare spanning verhoogd voor 3-assige druk volgens Eurocode 2
    alphacc = 1                                  
    sigma2 = sigmaEd1                           #horizontale spanning in knoop die steundruk levert, waardoor de druksterkte stijgt
    fckc = fck * (1.125 + 2.5 * (sigma2 / fck)) #EC2 vgl. 3.25
    gammac = 1.5                                #safety coefficient for concrete
    fcdc = alphacc * fckc / gammac              #EC2 vgl. 3.15
    k4 = 3                                      #recommended value EC2 art. 6.5.4 (6)
    v = 1 - fck / 250
    fcd = fck / gammac 
    sigmaRda = min([fcdc, k4 * v * fcd])
    return sigmaRda, fckc

def testA(fck, sigmaRda, sigmaEd0):  
    gammac = 1.5
    k4 = 3                                      #recommended value EC2 art. 6.5.4 (6)
    v = 1 - fck / 250
    fcd = fck / gammac 
    if k4 * v * fcd >= sigmaRda and sigmaRda >= sigmaEd0:
        return True
    else:
        return False

##### Knoop b

Knoop b bevindt zich boven de funderingspalen. De knoop wordt berekend als een CCT-knoop. De knoop ondervindt druk vanuit de kolom en de drukschoor, de trek wordt opgenomen door twee orthogonale wapeningsbanden. De kracht uit de diagonale staaf van het staafwerk wordt ontbonden naar deze orthogonale banden.



In [3]:
#Knoop b
def knoopB(Dp, ULS, Dia, z, theta, c, dstaaf):
    F0 = 0.25 * ULS
    F3 = ((0.5 * Dia - 150) / z) * F0
    F1 = np.sqrt(F0**2 + F3**2)
    a4 = Dp
    F4 = 0.25 * ULS
    a1b = a4 * np.sin(theta) + (dstaaf + 2 * (c + 0.5 * dstaaf)) * np.cos(theta)
    b1b = Dp + dstaaf + 2 * (c + 0.5 * dstaaf)
    
    sigmaEd4 = F4 * 1000 / (a4 * Dp)               #rekenwaarde spanning
    sigmaEd1b = F1 * 1000 / (a1b * b1b)
    return sigmaEd4, sigmaEd1b, a1b 

def maxB(fck):                                     #toelaatbare spanning
    k3 = 0.75                                      #recommended value EC art. 6.5.4 (4c)                                 
    v = 1 - fck / 250
    gammac = 1.5
    fcd = fck / gammac                                       
    sigmaRdb = k3 * v * fcd                        #EC2 vgl. 6.62
    return sigmaRdb

def testB(sigmaEd1b, sigmaRdb, sigmaEd4):                         
    if sigmaRdb >= sigmaEd1b and sigmaRdb >= sigmaEd4:                      #toetsing
        return True
    else:
        return False

##### Drukstaaf (1)
De spanning in de drukdiagonaal is automatisch berekend in knoop a. Deze is gelijk aan sigmaEd1a. De maximaal toelaatbare spanning in de drukdiagonaal staat hieronder.

In [38]:
def max1(fck):
    k1 = 1
    v = 1 - fck / 250
    gammac = 1.5
    fcd = fck / gammac
    sigmaRd1 = k1 * v * fcd
    return sigmaRd1

def teststaaf1(sigmaRd1, sigmaEd1):
    if sigmaRd1 >= sigmaEd1:
        return True
    else:
        return False

##### Trekstaaf (2)
De kracht in de trekstaaf wordt volledig openomen door de hoofdwapening. 

In [4]:
#Trekstaaf (2)
def staaf2(ULS, Dia, z):
    F0 = 0.25 * ULS
    F3 = ((0.5 * Dia - 150) / z) * F0
    F1 = np.sqrt(F0**2 + F3**2)
    F4 = 0.25 * ULS
    F2 = np.sqrt(F1**2 - F4**2)
    F2par = F2 / np.sqrt(2)                #De diagonale trekkracht in staaf 2 van het staafwerk wordt ontbonden naar twee orthogonae trekstangen
    return F2par

def max2(F2par, fsk):
    gammas = 1.15                          #veiligheidsfactor staal
    fsd = fsk / gammas
    Asreq = F2par * 1000 / fsd             #benodigde oppervlak wapening
    return Asreq

def Astoe(As):
    Atoe = As                              #toegepaste wapening
    return Atoe
    
def teststaaf2(Asreq, Atoe):
    if Atoe >= Asreq:
        return True
    else:
        return False

#### Scheurvorming

Ondanks dat de wapening de trekkrachten onder in de poer op zich neemt zal de rek er nog steeds voor zorgen dat er scheuren in het beton ontstaan. Deze scheurtjes mogen echter niet zo groot zijn dat de wapening bloot wordt gesteld aan milieu-invloeden. Daarom is het van belang de scheurwijdte te toetsen. De scheurwijdte kan positief worden beïnvloed door bijvoorbeeld meer wapening toe te passen of door de dekking te verhogen (als de dekking groter is dan 2 keer de nominale dekking mag de gunstige invloed niet meer inrekening worden gebracht). Voor de berekening van de scheurwijdte is EC2 art. 7.3.4 toegepast.

In [5]:
#scheurvorming
def scheur(c, dstaaf, SLS, ULS, F2par, fsk, b, fctm, Ecm, As): 
    #scheurwijdte
    ks1 = 0.8                                                       #EC vgl. 7.8 t/m 7.11
    ks2 = 1
    ks3 = 3.4
    ks4 = 0.425
    hcef = 2.5 * (c + dstaaf / 2)                                        
    Acef = b * hcef                                           
    rho = As / Acef
    srmax = ks3 * c + (ks1 * ks2 * ks4 * dstaaf) / rho
    
    gammas = 1.15                                                   #veiligheidsfactor staal
    fsd = fsk / gammas
    Asreq = F2par * 1000 / fsd                                      #benodigde oppervlak wapening                                               #toegepaste wapening
    sigmas = (0.25*SLS / (0.25*ULS)) * (Asreq / As) * fsd  
    kt = 0.4
    fcteff = fctm                                           
    Es = 2.0 * 10 ** 5                                      
    Em = Ecm * 1000                                         
    alphae = Es / Em
    e1 = (sigmas - kt * (fcteff / rho) * (1 + alphae * rho)) / Es   #esm - ecm
    e2 = 0.6 * (sigmas / Es)
    e = max([e1, e2])
       
    wk = srmax * e
    return wk

def maxscheur(c, cnom):
    kx = min([c / cnom, 2])
    wmax = 0.3 * kx                                         
    return wmax

#toetsing
def scheurtest(wk, wmax, c, cnom):
    kx = c / cnom
    if kx > 2:
        print('dekking is groter dan de waarde waarvoor gunstig effect in rekening mag worden gebracht') 
        
    if wk > wmax:
        return False
    else:
        return True

#### Tussenstaafafstand

De afstand tussen de wapeningsstaven heeft een minimale waarde. Als de staafafstand kleiner wordt dan de minimale staafafstand kan het voorkomen dat het beton zich tijdens het storten niet goed verdeelt onderin de poer, omdat het boven de staven blijft hangen.

In [6]:
#tussenstaafafstand
def tussenstaafafstand(Dp, O, c, dstaaf, n, b, dverdeel):
    a = (b - n * dstaaf) / (n-1)
    bmax = (0.5 * Dp + O - c - dstaaf - dverdeel) * 2
    k1t = 1                                                      #EC2 8.2(2)
    k2t = 5                                                      #EC2 8.2(2)
    dg = 32                                                      #max. aggregate size of concrete
    am = np.arange(3)
    am[0] = k1t * dstaaf
    am[1] = k2t + dg
    am[2] = 20
    amin = np.max(am)
    return a, amin, bmax

#toetsing
def afstandtest(a, b, amin, bmax):
    if a >= amin and bmax > b:
        return True
    else:
        return False

#### Verankering

In [30]:
#verankering
def bvl(F2par, fsk, dstaaf, As):             #basisverankeringslengte
    gammas = 1.15
    fsd = fsk / gammas
    Asreq = F2par * 1000 / fsd               #benodigde oppervlak wapening
    Atoe = As                                #toegepaste wapening
    sigmasd = (Asreq / Atoe) * fsd           #treksterkte wapening
    n1 = 1                                   #for good bonding conditions
    if dstaaf <= 32:
        n2 = 1
    else:
        n2 = (132 - dstaaf) / 100
    fctd = 1.35 #Treksterkte beton
    fbd = 2.25 * n1 * n2 * fctd              #EC2 vgl. 8.2
    lbrqd = 0.25 * dstaaf * (sigmasd / fbd)  #EC2 vgl. 8.3
    return lbrqd
    
def rvl(lbrqd, dstaaf):                      #rekenwaarde verankeringslengte
    alpha1 = 1 
    alpha2 = 1
    alpha3 = 1
    alpha4 = 1
    alpha5 = 1
    lb = alpha1 * alpha2 * alpha3 * alpha4 * alpha5 * lbrqd  #EC2 vgl. 8.4
    lbmin = 0.3 * lbrqd
    lbd = max([lb, lbmin, 10*dstaaf, 100]) 
    return lbd
        
#Toegepaste verankeringslengte
def verankering(c, dstaaf, Dp, d):
    s0 = c + dstaaf
    vl = Dp + 2 * s0 + 0.25 * np.pi * 6 * dstaaf + d
    return vl

def testverankering(lbd, vl, a , dstaaf, b, Dp): 
    if b <= Dp:
        lbdtest = lbd
    else:
        lbdtest = lbd + a + dstaaf #de staven die niet boven de poer liggen zijn maatgevend, hun verankering begint op een afstand van a + dstaaf achter de paal
    if vl >= lbdtest:                  
        return True
    else:
        return False

#Lengte van de poer voorbij de funderingspalen
def lvl(c, dstaaf, dverdeel, cnom):
    s0 = c + dstaaf
    dm = 5 * dstaaf
    lvl = 2 * s0 + 0.5 * dm + dstaaf + dverdeel + cnom #minimaal benodigde afstand tussen paal en zijvlak poer
    return lvl

#### Overige wapening
Flankwapening, bovenwapening en onderwapening.
Deze dient toegpast te worden om het beton op te sluiten en uitpuilen van het beton te voorkomen. De berekening van de hoeveelheden overige wapening is ontleent aan het rekenvoorbeeld uit 'EC2 in de praktijk; rekenvoorbeelden'.

In [31]:
def flankwapening(b, H, L, cnom):
    Asdbmin = 0.001 * b
    Vsfw = Asdbmin * H * 4 * (L - 2 * cnom)
    return Vsfw/ 1000000000

def bovenwapening(dstaaf, As, L, H, d, c):
    Asbw = 0.15 * 2 * As
    Vsbw = Asbw * 2 * (L - 2 * c + H - d - 2 * c)
    return Vsbw/ 1000000000

def onderwapening(b, L, hoh, vl):
    Asow = 0.001 * b
    Vsow = Asow * 2 * (hoh + 2 * vl) * (1 - 2 * b / L) 
    return Vsow/ 1000000000

def hoofdwapening(hoh, vl, As):
    Vshw = (hoh + 2 * vl) * 4 * As
    return Vshw/ 1000000000

#### Overige functies
Hieronder worden nog de materiaalhoeveelheden berkend, daarna worden twee randvoorwaarden voor de afmetingen van de poer getoetst en worden nog functies gedefinieerd voor het bepalen van de kosten en milieubelasting.

In [26]:
def betonvolume(H, L, B):
    Vb = H * L * L / 10**9
    return Vb

def staalmassa(Vshw, Vsfw, Vsbw, Vsow):
    Vs = (Vshw + Vsfw + Vsbw + Vsow) 
    rhos = 8000
    ms = Vs * rhos
    return ms

In [10]:
def lengte(H, Dia):
    if Dia / H <= 3:
        return True
    else:
        return False

In [13]:
def maxhef(h, a1a, a1b, theta):
    return h - 0.5 * a1a * np.cos(theta) - 0.5 * a1b * np.cos(theta)

def testhef(z, zmax):
    if z > zmax:
        return False
    else:
        return True

In [14]:
def kosten(Vb, ms, kb, ks):
    kosten = kb * Vb + ks * ms
    return kosten  

def duurzaamheid(Vb, ms, db, ds):
    rhob = 2500
    duurzaamheid = db * rhob * Vb + ds * ms
    return duurzaamheid

#### Berekening

Hieronder zijn alle bovenstaande functies twee keer samengevoegd in één functie. De functies hebben als input al de mogelijke parameters. De output van de functies zijn respectievelijk de kosten en de milieubelasting van de poer. Als de bepaalde configuratie van de poer niet voldoet voor de gegeven belastingssituatie, dan geeft de functie NaN als resultaat. Daarna volgt nog de controleberekening, deze lijkt op de andere, maar wordt uiteindelijk gebruikt om de uitkomst van de optimalisatie te controleren.

In [39]:
def berekening(Dp, Dk, H, dstaaf, n, dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c):
    hoh = 3 * Dp + 50
    Dia = np.sqrt(2 * hoh ** 2)                 #Diagonale h-o-h afstand van de palen (lengte van de trekstaaf in het staafwerkmodel)
    z = min([0.2 * Dia + 0.4 * H, 0.6 * Dia])   #Hefboomsarm
    theta = np.arctan(z / ((Dia/2)-0.25*Dk))
    As = n * 0.25 * np.pi * dstaaf**2
    
    SLS = Gk + Qk                               #service limit state
    ULS = Gk * GammaG + Qk * GammaQ             #Ultimate limit state
    
    sterkte = ['C12/15', 'C16/20', 'C20/25', 'C25/30', 'C30/37', 'C35/45', 'C40/50', 'C45/55', 'C50/60', 'C55/67', 'C60/75']
    fck = [12, 16, 20, 25, 30, 35, 40, 45, 50, 55, 60]
    fctm = [1.6, 1.9, 2.2, 2.6, 2.9, 3.2, 3.5, 3.8, 4.1, 4.2, 4.4]
    Ecm = [27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39]
    S = sterkte.index(beton)
    fck = fck[S]
    fctm = fctm[S]
    Ecm = Ecm[S]
    
    staalsterkte = ['B500B']
    fsk = [500]
    T = staalsterkte.index(staal)
    fsk = fsk[T]

    cnom = betondekking(dstaaf, constructieklasse)               #minimaal benodigde betondekking
    
    sigmaEd1, sigmaEd0, a1a = knoopA(Dk, ULS, theta)
    sigmaRda, fckc = maxA(sigmaEd1, fck)
    sigmaEd4, sigmaEd1b, a1b = knoopB(Dp, ULS, Dia, z, theta, c, dstaaf)
    sigmaRdb = maxB(fck)
    zmax = maxhef(H, a1a, a1b, theta)
    
    sigmaRd1 = max1(fck)
    
    F2par = staaf2(ULS, Dia, z)
    Asreq = max2(F2par, fsk)
    Atoe = Astoe(As)
    
    wk = scheur(c, dstaaf, SLS, ULS, F2par, fsk, b, fctm, Ecm, As)
    wmax = maxscheur(c, cnom)
    
    lbrqd = bvl(F2par, fsk, dstaaf, As)
    lbd = rvl(lbrqd, dstaaf)
    vl = verankering(c, dstaaf, Dp, d)
    O = lvl(c, dstaaf, dverdeel, cnom)
    a, amin, bmax = tussenstaafafstand(Dp, O, c, dstaaf, n, b, dverdeel)
    
    L = Dp + hoh + 2 * O
    B = L
    
    Vsfw = flankwapening(b, H, L, cnom)
    Vsbw = bovenwapening(dstaaf, As, L, H, d, c)
    Vsow = onderwapening(b, L, hoh, vl)
    Vshw = hoofdwapening(hoh, vl, As)
    
    Vb = betonvolume(H, L, L)
    ms = staalmassa(Vshw, Vsfw, Vsbw, Vsow)
    
    cost = kosten(Vb, ms, kb, ks)
    duur = duurzaamheid(Vb, ms, db, ds)
    
    t = np.zeros(10)
    t[0] = testdekking(c, cnom, dstaaf)     
    t[1] = testA(fck, sigmaRda, sigmaEd0)
    t[2] = testB(sigmaEd1b, sigmaRdb, sigmaEd4)
    t[3] = teststaaf2(Asreq, Atoe)
    t[4] = scheurtest(wk, wmax, c, cnom)
    t[5] = afstandtest(a, b, amin, bmax)
    t[6] = testverankering(lbd, vl, a , dstaaf, b, Dp)
    t[7] = lengte(H, Dia)
    t[8] = testhef(z, zmax)
    t[9] = teststaaf1(sigmaRd1, sigmaEd1)
    
    if np.all(t == True):
        return cost
    else:
        return np.NaN 
    
def berekening1(Dp, Dk, H, dstaaf, n, dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c):
    hoh = 3 * Dp + 50
    Dia = np.sqrt(2 * hoh ** 2)                 #Diagonale h-o-h afstand van de palen (lengte van de trekstaaf in het staafwerkmodel)
    z = min([0.2 * Dia + 0.4 * H, 0.6 * Dia])   #Hefboomsarm
    theta = np.arctan(z / ((Dia/2)-0.25*Dk))
    As = n * 0.25 * np.pi * dstaaf**2
    
    SLS = Gk + Qk                               #service limit state
    ULS = Gk * GammaG + Qk * GammaQ             #Ultimate limit state
    
    sterkte = ['C12/15', 'C16/20', 'C20/25', 'C25/30', 'C30/37', 'C35/45', 'C40/50', 'C45/55', 'C50/60', 'C55/67', 'C60/75']
    fck = [12, 16, 20, 25, 30, 35, 40, 45, 50, 55, 60]
    fctm = [1.6, 1.9, 2.2, 2.6, 2.9, 3.2, 3.5, 3.8, 4.1, 4.2, 4.4]
    Ecm = [27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39]
    S = sterkte.index(beton)
    fck = fck[S]
    fctm = fctm[S]
    Ecm = Ecm[S]
    
    staalsterkte = ['B500B']
    fsk = [500]
    T = staalsterkte.index(staal)
    fsk = fsk[T]

    cnom = betondekking(dstaaf, constructieklasse)               #minimaal benodigde betondekking
    
    sigmaEd1, sigmaEd0, a1a = knoopA(Dk, ULS, theta)
    sigmaRda, fckc = maxA(sigmaEd1, fck)
    sigmaEd4, sigmaEd1b, a1b = knoopB(Dp, ULS, Dia, z, theta, c, dstaaf)
    sigmaRdb = maxB(fck)
    zmax = maxhef(H, a1a, a1b, theta)
    
    sigmaRd1 = max1(fck)
    
    F2par = staaf2(ULS, Dia, z)
    Asreq = max2(F2par, fsk)
    Atoe = Astoe(As)
    
    wk = scheur(c, dstaaf, SLS, ULS, F2par, fsk, b, fctm, Ecm, As)
    wmax = maxscheur(c, cnom)
    
    lbrqd = bvl(F2par, fsk, dstaaf, As)
    lbd = rvl(lbrqd, dstaaf)
    vl = verankering(c, dstaaf, Dp, d)
    O = lvl(c, dstaaf, dverdeel, cnom)
    a, amin, bmax = tussenstaafafstand(Dp, O, c, dstaaf, n, b, dverdeel)
    
    L = Dp + hoh + 2 * O
    B = L
    
    Vsfw = flankwapening(b, H, L, cnom)
    Vsbw = bovenwapening(dstaaf, As, L, H, d, c)
    Vsow = onderwapening(b, L, hoh, vl)
    Vshw = hoofdwapening(hoh, vl, As)
    
    Vb = betonvolume(H, L, L)
    ms = staalmassa(Vshw, Vsfw, Vsbw, Vsow)
    
    cost = kosten(Vb, ms, kb, ks)
    duur = duurzaamheid(Vb, ms, db, ds)
    
    t = np.zeros(10)
    t[0] = testdekking(c, cnom, dstaaf)     
    t[1] = testA(fck, sigmaRda, sigmaEd0)
    t[2] = testB(sigmaEd1b, sigmaRdb, sigmaEd4)
    t[3] = teststaaf2(Asreq, Atoe)
    t[4] = scheurtest(wk, wmax, c, cnom)
    t[5] = afstandtest(a, b, amin, bmax)
    t[6] = testverankering(lbd, vl, a , dstaaf, b, Dp)
    t[7] = lengte(H, Dia)
    t[8] = testhef(z, zmax)
    t[9] = teststaaf1(sigmaRd1, sigmaEd1)
    
    if np.all(t == True):
        return duur
    else:
        return np.NaN  

In [40]:
def controleberekening(Dp, Dk, H, dstaaf, n, dverdeel, b, Gk, Qk, GammaG, GammaQ, constructieklasse, d, beton, staal, kb, ks, db, ds, c):
    hoh = 3 * Dp + 50
    Dia = np.sqrt(2 * hoh ** 2)                 #Diagonale h-o-h afstand van de palen (lengte van de trekstaaf in het staafwerkmodel)
    z = min([0.2 * Dia + 0.4 * H, 0.6 * Dia])   #Hefboomsarm
    theta = np.arctan(z / ((Dia/2)-0.25*Dk))
    As = n * 0.25 * np.pi * dstaaf**2
    
    SLS = Gk + Qk                               #service limit state
    ULS = Gk * GammaG + Qk * GammaQ             #Ultimate limit state
    
    sterkte = ['C12/15', 'C16/20', 'C20/25', 'C25/30', 'C30/37', 'C35/45', 'C40/50', 'C45/55', 'C50/60', 'C55/67', 'C60/75']
    fck = [12, 16, 20, 25, 30, 35, 40, 45, 50, 55, 60]
    fctm = [1.6, 1.9, 2.2, 2.6, 2.9, 3.2, 3.5, 3.8, 4.1, 4.2, 4.4]
    Ecm = [27, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39]
    S = sterkte.index(beton)
    fck = fck[S]
    fctm = fctm[S]
    Ecm = Ecm[S]
    
    staalsterkte = ['B500B']
    fsk = [500]
    T = staalsterkte.index(staal)
    fsk = fsk[T]

    cnom = betondekking(dstaaf, constructieklasse)               #minimaal benodigde betondekking
    
    sigmaEd1, sigmaEd0, a1a = knoopA(Dk, ULS, theta)
    sigmaRda, fckc = maxA(sigmaEd1, fck)
    sigmaEd4, sigmaEd1b, a1b = knoopB(Dp, ULS, Dia, z, theta, c, dstaaf)
    sigmaRdb = maxB(fck)
    zmax = maxhef(H, a1a, a1b, theta)
    
    sigmaRd1 = max1(fck)
    
    F2par = staaf2(ULS, Dia, z)
    Asreq = max2(F2par, fsk)
    Atoe = Astoe(As)
    
    wk = scheur(c, dstaaf, SLS, ULS, F2par, fsk, b, fctm, Ecm, As)
    wmax = maxscheur(c, cnom)
    
    lbrqd = bvl(F2par, fsk, dstaaf, As)
    lbd = rvl(lbrqd, dstaaf)
    vl = verankering(c, dstaaf, Dp, d)
    O = lvl(c, dstaaf, dverdeel, cnom)
    a, amin, bmax = tussenstaafafstand(Dp, O, c, dstaaf, n, b, dverdeel)
    
    L = Dp + hoh + 2 * O
    B = L
    
    Vsfw = flankwapening(b, H, L, cnom)
    Vsbw = bovenwapening(dstaaf, As, L, H, d, c)
    Vsow = onderwapening(b, L, hoh, vl)
    Vshw = hoofdwapening(hoh, vl, As)
    
    Vb = betonvolume(H, L, L)
    ms = staalmassa(Vshw, Vsfw, Vsbw, Vsow)
    
    prijs = kosten(Vb, ms, kb, ks)
    milieu = duurzaamheid(Vb, ms, db, ds)
    
    t = np.zeros(10)
    t[0] = testdekking(c, cnom, dstaaf)     
    t[1] = testA(fck, sigmaRda, sigmaEd0)
    t[2] = testB(sigmaEd1b, sigmaRdb, sigmaEd4)
    t[3] = teststaaf2(Asreq, Atoe)
    t[4] = scheurtest(wk, wmax, c, cnom)
    t[5] = afstandtest(a, b, amin, bmax)
    t[6] = testverankering(lbd, vl, a , dstaaf, b, Dp)
    t[7] = lengte(H, Dia)
    t[8] = testhef(z, zmax)
    t[9] = teststaaf1(sigmaRd1, sigmaEd1)
    
    if np.all(t == True):
        return L, Vb, O, Vshw, Vsfw, Vsbw, Vsow, ms, a, b, c, d, SLS, ULS, hoh, Dia, z, theta, cnom, sigmaEd1, sigmaEd0, sigmaRda, sigmaEd4, sigmaEd1b, sigmaRdb, F2par, Asreq, Atoe, wk, wmax, lbd, vl, prijs, milieu, amin, bmax, fckc, sigmaRd1
    elif t[0] == False:
        print('betondekking onvoldoende')
    elif t[1] == False:
        print('betonspanning in knoop a voldoet niet')
    elif t[2] == False:
        print('betonspanning in knoop b voldoet niet')
    elif t[3] == False:
        print('spanning in de wapeningsstaven voldoet niet')
    elif t[4] == False:
        print('scheurwijdte voldoet niet')
    elif t[5] == False:
        print('staafafstand voldoet niet')
    elif t[6] == False:
        print('verankeringslengte voldoet niet')
    elif t[7] == False:
        print('hoogte te klein, poer is geen gedrongen element')
    elif t[8] == False:
        print('Inwendige hefboomsarm is te groot voor door de hoogte van de poer')
    elif t[9] == False:
        print('spanning in drukschoor (1) voldoet niet')